In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "../outputs/module1_results.csv"
df = pd.read_csv(path)
df

,image_name,vehicle_count,Pedestrian_count,congestion
0,7d06fefd-f7be05a6.jpg,9,0,LOW
1,7d128593-0ccfea4c.jpg,3,0,LOW
2,7d15b18b-1e0d6e3f.jpg,6,0,LOW
3,7d209219-ccdc1a09.jpg,5,0,LOW
4,7d22891c-224788c0.jpg,3,0,LOW
...,...,...,...,...
995,abc6b442-311adbdf.jpg,2,0,LOW
996,abf6f644-ade3cfa2.jpg,5,0,LOW
997,abfbfcb1-0e220001.jpg,17,0,Medium
998,ac033a32-493865fc.jpg,10,1,Medium


In [3]:
feature = df[["vehicle_count", "Pedestrian_count"]].values
feature

array([[ 9,  0],
       [ 3,  0],
       [ 6,  0],
       ...,
       [17,  0],
       [10,  1],
       [10,  3]], shape=(1000, 2))

In [4]:
# Normalize features
feature = (feature - feature.mean(axis=0)) / feature.std(axis=0)
feature

array([[ 0.56070817, -0.39291135],
       [-0.88690328, -0.39291135],
       [-0.16309756, -0.39291135],
       ...,
       [ 2.49085675, -0.39291135],
       [ 0.80197674,  0.35549123],
       [ 0.80197674,  1.85229638]], shape=(1000, 2))

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Model

In [6]:
class VAE(Model):
    def __init__(self, input_dim, latent_dim=2):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = tf.keras.Sequential(
            [layers.Dense(16, activation="relu"), layers.Dense(8, activation="relu")]
        )
        self.z_mean = layers.Dense(latent_dim)
        self.z_log_var = layers.Dense(latent_dim)

        # Decoder
        self.decoder = tf.keras.Sequential(
            [
                layers.Dense(16, activation="relu"),
                layers.Dense(8, activation="relu"),
                layers.Dense(input_dim),
            ]
        )

    def sample(self, z_mean, z_log_var):
        eps = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * eps

    def call(self, inputs):
        encode = self.encoder(inputs)
        zmean = self.z_mean(encode)
        zlog_var = self.z_log_var(encode)
        z = self.sample(zmean, zlog_var)
        recontrucion = self.decoder(z)

        kl_loss = -0.5 * tf.reduce_mean(
            1 + zlog_var - tf.square(zmean) - tf.exp(zlog_var)
        )
        self.add_loss(kl_loss)

        return recontrucion

In [7]:
vae = VAE(input_dim=feature.shape[1])
vae.compile(optimizer="adam", loss="mse")

In [8]:
vae.fit(feature, feature, epochs=30, batch_size=32)

Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0843
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0383 
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0212 
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0135 
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9937 
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9812 
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9686 
Epoch 8/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9708 
Epoch 9/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9405 
Epoch 10/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9488 
Epoch 11/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9069 
Epoch 12/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9344 
Epoch 13/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8817 
Epoch 14/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8626
Epoch 15/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8148 
Epoch 

In [9]:
reconstructed = vae.predict(feature)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [10]:
reconstructed_error = np.mean(np.square(feature - reconstructed), axis=1)
reconstructed_error

array([1.64801061e-01, 3.12035282e-02, 3.36959171e-02, 6.57880124e-03,
       8.36770876e-02, 1.95429079e-01, 7.71287174e-01, 8.12077199e-01,
       2.17853612e-01, 2.81160712e-01, 8.81889564e-01, 6.42118351e-01,
       9.34486576e-02, 5.94713926e-01, 1.73342791e-01, 3.34931743e-01,
       1.14385371e+00, 2.63682453e-01, 2.18876015e-01, 1.11966533e-01,
       1.82716305e-01, 8.98287450e-02, 5.79542876e-02, 1.42189805e-01,
       6.88409261e-03, 4.19216760e-02, 2.61631833e-01, 7.72015217e-01,
       2.41516250e-01, 1.18877644e+00, 9.78219441e-02, 1.09634476e-02,
       5.32283830e-01, 1.41332886e-02, 3.91000941e-01, 7.77707443e-01,
       1.11330908e+00, 1.68697058e-01, 1.40701833e-01, 3.66947630e-01,
       7.18148008e-01, 4.95651177e-01, 1.42608486e-02, 2.34167498e-01,
       2.80865004e-01, 1.29769771e-01, 1.86316979e-01, 2.90672213e-01,
       1.05906882e+00, 1.12873362e+00, 1.32385469e+00, 4.73514379e-03,
       2.71287232e-01, 4.02241952e-01, 2.63078847e-02, 7.53858223e-03,
      

In [11]:
threshold = np.percentile(reconstructed_error, 95)
threshold

np.float64(1.43501140604778)

In [12]:
numpy_arr = np.array(reconstructed_error)
for i in numpy_arr:
    print(i)

0.16480106094265218
0.031203528184357995
0.03369591709932318
0.006578801239498316
0.08367708764020362
0.19542907853841915
0.7712871736223971
0.8120771987380019
0.21785361198011965
0.2811607118863661
0.881889564472832
0.6421183505705051
0.09344865755394877
0.5947139260703211
0.17334279061997748
0.33493174292634537
1.1438537093351486
0.2636824525496079
0.21887601487710373
0.11196653295325301
0.18271630450976883
0.08982874497123974
0.05795428758866635
0.14218980498575473
0.006884092607563308
0.041921676048400594
0.2616318329095359
0.7720152166934947
0.24151625042027952
1.1887764394347198
0.09782194406901476
0.010963447602988247
0.5322838296444589
0.014133288576538186
0.39100094116213935
0.7777074433149357
1.1133090754759207
0.16869705796895065
0.14070183286663568
0.3669476300277643
0.7181480081286407
0.4956511772596721
0.014260848578547551
0.23416749753761895
0.2808650042170665
0.1297697708965848
0.18631697909930145
0.2906722132306881
1.0590688196546125
1.1287336154516194
1.32385469464955

In [13]:
anomaly = reconstructed_error > threshold
anomaly.sum()

np.int64(50)